In [0]:
import datetime
users = [
    {
        "id": 1,
        "first_name": "Corrie",
        "last_name": "Van den Oord",
        "email": "cvandenoord0@etsy.com",
        "phone_numbers": {"mobile": "+1 234 567 8901", "home": "+1 234 567 8911"},
        "is_customer": True,
        "amount_paid": 1000.55,
        "customer_from": datetime.date(2021, 1, 15),
        "last_updated_ts": datetime.datetime(2021, 2, 10, 1, 15, 0)
    },
    {
        "id": 2,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "nbrewitt1@dailymail.co.uk",
        "phone_numbers": {"mobile": "+1 234 567 8923", "home": "+1 234 567 8934"},
        "is_customer": True,
        "amount_paid": 900.0,
        "customer_from": datetime.date(2021, 2, 14),
        "last_updated_ts": datetime.datetime(2021, 2, 18, 3, 33, 0)
    },
    {
        "id": 3,
        "first_name": "Orelie",
        "last_name": "Penney",
        "email": "openney2@vistaprint.com",
        "phone_numbers": {"mobile": "+1 714 512 9752", "home": "+1 714 512 6601"},
        "is_customer": True,
        "amount_paid": 850.55,
        "customer_from": datetime.date(2021, 1, 21),
        "last_updated_ts": datetime.datetime(2021, 3, 15, 15, 16, 55)
    },
    {
        "id": 4,
        "first_name": "Ashby",
        "last_name": "Maddocks",
        "email": "amaddocks3@home.pl",
        "phone_numbers": None,
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 10, 17, 45, 30)
    },
    {
        "id": 5,
        "first_name": "Kurt",
        "last_name": "Rome",
        "email": "krome4@shutterfly.com",
        "phone_numbers": {"mobile": "+1 817 934 7142"},
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 2, 0, 55, 18)
    }
]

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, explode, explode_outer
usersDf=spark.createDataFrame([Row(**user) for user in users])
display(usersDf)

id,first_name,last_name,email,phone_numbers,is_customer,amount_paid,customer_from,last_updated_ts
1,Corrie,Van den Oord,cvandenoord0@etsy.com,"Map(mobile -> +1 234 567 8901, home -> +1 234 567 8911)",true,1000.55,2021-01-15,2021-02-10T01:15:00.000+0000
2,Nikolaus,Brewitt,nbrewitt1@dailymail.co.uk,"Map(mobile -> +1 234 567 8923, home -> +1 234 567 8934)",true,900.0,2021-02-14,2021-02-18T03:33:00.000+0000
3,Orelie,Penney,openney2@vistaprint.com,"Map(mobile -> +1 714 512 9752, home -> +1 714 512 6601)",true,850.55,2021-01-21,2021-03-15T15:16:55.000+0000
4,Ashby,Maddocks,amaddocks3@home.pl,null,false,null,null,2021-04-10T17:45:30.000+0000
5,Kurt,Rome,krome4@shutterfly.com,Map(mobile -> +1 817 934 7142),false,null,null,2021-04-02T00:55:18.000+0000


In [0]:
usersDf.dtypes

Out[3]: [('id', 'bigint'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('email', 'string'),
 ('phone_numbers', 'map<string,string>'),
 ('is_customer', 'boolean'),
 ('amount_paid', 'double'),
 ('customer_from', 'date'),
 ('last_updated_ts', 'timestamp')]

In [0]:
usersDf.select("id", col("phone_numbers")['mobile'].alias("mobile"), col("phone_numbers")['home'].alias("home")).show(truncate=False)

+---+---------------+---------------+
|id |mobile         |home           |
+---+---------------+---------------+
|1  |+1 234 567 8901|+1 234 567 8911|
|2  |+1 234 567 8923|+1 234 567 8934|
|3  |+1 714 512 9752|+1 714 512 6601|
|4  |null           |null           |
|5  |+1 817 934 7142|null           |
+---+---------------+---------------+



In [0]:
usersDf.select("id", explode("phone_numbers")).show(truncate=False)

+---+------+---------------+
|id |key   |value          |
+---+------+---------------+
|1  |mobile|+1 234 567 8901|
|1  |home  |+1 234 567 8911|
|2  |mobile|+1 234 567 8923|
|2  |home  |+1 234 567 8934|
|3  |mobile|+1 714 512 9752|
|3  |home  |+1 714 512 6601|
|5  |mobile|+1 817 934 7142|
+---+------+---------------+



In [0]:
usersDf \
    .select("id", explode("phone_numbers")) \
    .withColumnRenamed("key", "phone_type") \
    .withColumnRenamed("value","phone_number") \
    .show(truncate=False)

+---+----------+---------------+
|id |phone_type|phone_number   |
+---+----------+---------------+
|1  |mobile    |+1 234 567 8901|
|1  |home      |+1 234 567 8911|
|2  |mobile    |+1 234 567 8923|
|2  |home      |+1 234 567 8934|
|3  |mobile    |+1 714 512 9752|
|3  |home      |+1 714 512 6601|
|5  |mobile    |+1 817 934 7142|
+---+----------+---------------+



In [0]:
usersDf \
    .select("id", explode_outer("phone_numbers")) \
    .withColumnRenamed("key", "phone_type") \
    .withColumnRenamed("value","phone_number") \
    .show(truncate=False)

+---+----------+---------------+
|id |phone_type|phone_number   |
+---+----------+---------------+
|1  |mobile    |+1 234 567 8901|
|1  |home      |+1 234 567 8911|
|2  |mobile    |+1 234 567 8923|
|2  |home      |+1 234 567 8934|
|3  |mobile    |+1 714 512 9752|
|3  |home      |+1 714 512 6601|
|4  |null      |null           |
|5  |mobile    |+1 817 934 7142|
+---+----------+---------------+

